In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pandas as pd
from keras.preprocessing.text import Tokenizer
df = pd.read_csv('./events.csv')


events = df['event']
dates = df['date'].astype(int)

# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(events)
total_events = len(tokenizer.word_index) + 1


In [ ]:

events = df['event']
dates = df['date'].astype(int)

In [ ]:
sequences = []
for i, event in enumerate(events):
    token_list = tokenizer.texts_to_sequences([event])[0]
    sequences.append((token_list, dates[i]))

In [ ]:
max_event_length = max([len(seq[0]) for seq in sequences])
padded_sequences = []
for seq, date in sequences:
    padded_event = pad_sequences([seq], maxlen=max_event_length, padding='pre')[0]
    padded_sequences.append((padded_event, date))

In [ ]:
# Create predictors and labels
X = np.array([seq for seq, _ in padded_sequences])
y = np.array([date for _, date in padded_sequences])

In [ ]:
X[0]

In [ ]:
# Define and compile the model
model = Sequential()
model.add(Embedding(total_events, 100, input_length=max_event_length))
model.add(LSTM(150))
model.add(Dense(1,activation='relu'))
model.compile(loss='mae', optimizer='adam')

# Train the model
model.fit(X, y, epochs=5000, verbose=1)

In [ ]:
model.save_weights('./event-predictor-2880epoch.h5')
import pickle

# Assuming tokenizer is your Tokenizer object
# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
import pickle
# Load the tokenizer
with open('./tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)



In [ ]:
model = Sequential()
model.add(Embedding(total_events, 100, input_length=max_event_length))
model.add(LSTM(150))
model.add(Dense(1,activation='relu'))
model.load_weights('./event-predictor-2880epoch.h5')

In [ ]:
 # Function to generate event based on seed text and date
def generate_event(seed_text, event_date, model, max_event_length, tokenizer):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    padded_event = pad_sequences([token_list], maxlen=max_event_length, padding='pre')
    predicted_date = model.predict(padded_event)[0][0]
    return predicted_date

# Example usage
seed_event = "The egyptian revulation"
predicted_date = generate_event(seed_event, seed_date, model, max_event_length, tokenizer)
print("Predicted date for {}: {}".format(seed_event, int(predicted_date)))

In [ ]:
# Example usage
seed_event = "the arabian revulation"
seed_date = 2050
predicted_date = generate_event(seed_event, seed_date, model, max_event_length, tokenizer)
print("Predicted date for {}: {}".format(seed_event, int(predicted_date)))

In [ ]:
seed_event = "the world war III" 
seed_date = 2050
predicted_date = generate_event(seed_event, seed_date, model, max_event_length, tokenizer)
print("Predicted date for {}: {}".format(seed_event, int(predicted_date)))

In [ ]:
'we Are at year 1742 and accourding to the cyclye of the dates 1811 will be 2095'
'we Are at year 1742 and accourding to the cyclye of the dates 1450 will be 2029-2030'